<a href="https://colab.research.google.com/github/anirbanmukherjee2709/tsai_end2.0_Session_7/blob/master/Quora_Seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy import data
from torchtext.legacy.data import Field, BucketIterator


import spacy
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import time

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)


In [13]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
#!pip install spacy --upgrade

In [14]:
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

--2021-06-24 14:37:32--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.1.2, 151.101.65.2, 151.101.129.2, ...
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.1.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58176133 (55M) [text/tab-separated-values]
Saving to: ‘quora_duplicate_questions.tsv.1’

quora_duplicate_que 100%[===================>]  55.48M  80.1MB/s    in 0.7s    

2021-06-24 14:37:32 (80.1 MB/s) - ‘quora_duplicate_questions.tsv.1’ saved [58176133/58176133]



In [15]:
import pandas as pd
df = pd.read_csv("/content/quora_duplicate_questions.tsv", sep='\t', engine="python",error_bad_lines=False)

In [16]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0
2,2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0
3,3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0
4,4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0


In [17]:
df_subset = df.loc[:, ['question1', 'question2', 'is_duplicate']]
df_subset = df_subset[df_subset['is_duplicate'] == 1]
df_subset.reset_index(drop=True, inplace=True)


df_subset['question1'] = df_subset['question1'].astype(str)
df_subset['question2'] = df_subset['question2'].astype(str)
df_subset = df_subset.dropna()
df_subset.to_csv("quora_dataset.csv")

print("Number of records : ",len(df_subset))
print(df_subset[['question1','question2']].head(n = 20))

Number of records :  149263
                                                                                                                                      question1                                                                                    question2
0                                                        Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?   I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?
1                                                                                                                How can I be a good geologist?                                                    What should I do to be a great geologist?
2                                                                                                   How do I read and find my YouTube comments?                                                       How can I see all my Youtube comments?
3                       

In [18]:
%%bash
python -m spacy download en
#python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [19]:
#spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [20]:
'''def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]'''

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text.lower() for tok in spacy_en.tokenizer(text)]

In [21]:
question1 = Field(sequential=True,
                 #batch_first = True,
                 tokenize = tokenize_en, 
                 init_token = '<sos>', 
                 eos_token = '<eos>')


question2 = Field(sequential=True,
                 #batch_first = True,
                 tokenize = tokenize_en, 
                 init_token = '<sos>', 
                 eos_token = '<eos>')

fields = [('question1', question1),('question2',question2)]

In [22]:
example = [data.Example.fromlist([df_subset.question1.iloc[i],df_subset.question2.iloc[i]], fields) for i in range(df_subset.shape[0])] 
train_data, test_data = data.Dataset(example, fields).split(split_ratio = 0.7)

In [23]:
print(f"Number of training examples: {len(train_data.examples)}")
#print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 104484
Number of testing examples: 44779


In [24]:
print(vars(train_data.examples[0]))

{'question1': ['what', 'are', 'the', 'most', 'intellectually', 'stimulating', 'movies', 'you', 'have', 'ever', 'seen', '?'], 'question2': ['what', 'are', 'the', 'most', 'intellectually', 'stimulating', 'films', 'you', 'have', 'ever', 'watched', '?']}


In [25]:
question1.build_vocab(train_data, min_freq = 2)
question2.build_vocab(train_data, min_freq = 2)

print('Top 10 words in question1 Vocab :', list(question1.vocab.freqs.most_common(10)))

import os, pickle
with open('quora_question1_tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(question1.vocab.stoi, tokens)

  
with open('quora_question2_tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(question2.vocab.stoi, tokens)

Top 10 words in question1 Vocab : [('?', 108748), ('the', 46721), ('what', 41081), ('is', 31957), ('how', 31727), ('i', 26415), ('to', 23391), ('do', 22943), ('in', 20321), ('a', 19731)]


In [26]:
print(f"Unique tokens in source (question1) vocabulary: {len(question1.vocab)}")
print(f"Unique tokens in target (question2) vocabulary: {len(question2.vocab)}")

Unique tokens in source (question1) vocabulary: 14522
Unique tokens in target (question2) vocabulary: 14469


In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Curent Device is ", device)

Curent Device is  cuda


In [28]:
BATCH_SIZE = 128

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.question1),
    sort_within_batch=True,
    device = device)

In [29]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [30]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [31]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [32]:
INPUT_DIM = len(question1.vocab)
OUTPUT_DIM = len(question2.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [33]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(14522, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(14469, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=14469, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [34]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 22,200,709 trainable parameters


In [35]:
optimizer = optim.Adam(model.parameters(),lr=0.0005)

In [36]:
question2_PAD_IDX = question2.vocab.stoi[question2.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = question2_PAD_IDX)

In [37]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        ques1 = batch.question1
        ques2 = batch.question2
        
        optimizer.zero_grad()
        
        output = model(ques1, ques2)
        
        #ans = [ans len, batch size]
        #output = [ans len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        ques2 = ques2[1:].view(-1)
        
        #ans = [(ans len - 1) * batch size]
        #output = [(ans len - 1) * batch size, output dim]
        
        loss = criterion(output, ques2)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [38]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            ques1 = batch.question1
            ques2 = batch.question2

            output = model(ques1, ques2, 0) #turn off teacher forcing

            #ans = [ans len, batch size]
            #output = [ans len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            ques2 = ques2[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, ques2)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [39]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [40]:
N_EPOCHS = 20
CLIP = 1

best_test_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    test_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'quora-qa-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Test. Loss: {test_loss:.3f} |  Test. PPL: {math.exp(test_loss):7.3f}')

Epoch: 01 | Time: 7m 26s
	Train Loss: 5.010 | Train PPL: 149.934
	 Test. Loss: 4.784 |  Test. PPL: 119.636
Epoch: 02 | Time: 7m 27s
	Train Loss: 4.135 | Train PPL:  62.511
	 Test. Loss: 4.358 |  Test. PPL:  78.064
Epoch: 03 | Time: 7m 29s
	Train Loss: 3.653 | Train PPL:  38.581
	 Test. Loss: 4.074 |  Test. PPL:  58.791
Epoch: 04 | Time: 7m 28s
	Train Loss: 3.329 | Train PPL:  27.912
	 Test. Loss: 3.914 |  Test. PPL:  50.074
Epoch: 05 | Time: 7m 30s
	Train Loss: 3.090 | Train PPL:  21.975
	 Test. Loss: 3.773 |  Test. PPL:  43.499
Epoch: 06 | Time: 7m 30s
	Train Loss: 2.902 | Train PPL:  18.203
	 Test. Loss: 3.695 |  Test. PPL:  40.262
Epoch: 07 | Time: 7m 26s
	Train Loss: 2.747 | Train PPL:  15.594
	 Test. Loss: 3.667 |  Test. PPL:  39.134
Epoch: 08 | Time: 7m 26s
	Train Loss: 2.636 | Train PPL:  13.957
	 Test. Loss: 3.600 |  Test. PPL:  36.587
Epoch: 09 | Time: 7m 27s
	Train Loss: 2.514 | Train PPL:  12.355
	 Test. Loss: 3.592 |  Test. PPL:  36.320
Epoch: 10 | Time: 7m 25s
	Train Loss:

In [41]:
model.load_state_dict(torch.load('quora-qa-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.519 | Test PPL:  33.762 |


In [42]:
id = 2

example = train_data.examples[id]
print('question1: ', ' '.join(example.question1))
print('question2: ', ' '.join(example.question2))

question1:  how can i find my all gmail accounts ?
question2:  how can i find all my old gmail accounts ?


In [43]:
ques1_tensor = question1.process([example.question1]).to(device)
ques2_tensor = question2.process([example.question2]).to(device)
print(ques2_tensor.shape)

model.eval()
with torch.no_grad():
    outputs = model(ques1_tensor, ques2_tensor, teacher_forcing_ratio=0)

print(outputs.shape)
output_idx = outputs[1:].squeeze(1).argmax(1)
' '.join([question2.vocab.itos[idx] for idx in output_idx])

torch.Size([12, 1])
torch.Size([12, 1, 14469])


'how do i find all of gmail accounts ? <eos> <eos>'